In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm 

# 1. Load etfdata_2018.sas7bdat file.

In [0]:
etf = pd.read_sas('etfdata_2018.sas7bdat')
etf.head()

,PERMNO,DATE,COMNAM,TICKER,SHRCD,PRC,VOL,RET,SHROUT
0,10113.0,2018-01-02,b'ADVISORSHARES TRUST',b'AADR',73.0,60.299999,73052.0,0.024639,2850.0
1,10113.0,2018-01-03,b'ADVISORSHARES TRUST',b'AADR',73.0,60.930000,44385.0,0.010448,2850.0
2,10113.0,2018-01-04,b'ADVISORSHARES TRUST',b'AADR',73.0,61.160000,104484.0,0.003775,2850.0
3,10113.0,2018-01-05,b'ADVISORSHARES TRUST',b'AADR',73.0,61.639999,73198.0,0.007848,2850.0
4,10113.0,2018-01-08,b'ADVISORSHARES TRUST',b'AADR',73.0,61.639999,64271.0,0.000000,2850.0


#2. Drop 'COMNAM' and 'SHRCD' columns.

In [0]:
etf.drop(columns=['COMNAM', 'SHRCD'], inplace=True)
etf.head()

,PERMNO,DATE,TICKER,PRC,VOL,RET,SHROUT
0,10113.0,2018-01-02,b'AADR',60.299999,73052.0,0.024639,2850.0
1,10113.0,2018-01-03,b'AADR',60.930000,44385.0,0.010448,2850.0
2,10113.0,2018-01-04,b'AADR',61.160000,104484.0,0.003775,2850.0
3,10113.0,2018-01-05,b'AADR',61.639999,73198.0,0.007848,2850.0
4,10113.0,2018-01-08,b'AADR',61.639999,64271.0,0.000000,2850.0


# 3.  For every date, compute the mean return.

In [0]:
# Method 1
etf1 = etf.set_index(['DATE', 'PERMNO'])
etf1.mean(level='DATE').drop(columns = ['VOL', 'PRC', 'SHROUT']).head(10)

# Method 2
# etf1 = etf.set_index(['DATE', 'PERMNO'])
# etf1['RET'].mean(level='DATE').head()

# Method 3
# etf1 = etf.set_index(['PERMNO','DATE'])
# etf1.groupby('DATE').RET.mean()

,RET
DATE,
2018-01-02,0.005924
2018-01-03,0.004253
2018-01-04,0.003588
2018-01-05,0.003417
2018-01-08,0.001577
2018-01-09,-0.000389
2018-01-10,-0.002022
2018-01-11,0.006122
2018-01-12,0.005083


# 4.  To see what the groupby is doing, use a multi-index to organize the data by ‘PERMNO’ (permanent number is the unique identifier of a given ETF) and ‘DATE’ (hint: use .set_index()). In particular, we want the first index to be date, and the second to be the permanent number. Make sure the dates are in chronological order (hint: use .sort_index()).

In [0]:
etf1.sort_index(inplace=True)

etf1

TICKER        PRC       VOL       RET   SHROUT
DATE       PERMNO                                                  
2018-01-02 10113.0  b'AADR'  60.299999   73052.0  0.024639   2850.0
           11182.0  b'GASL'  27.610001  185904.0  0.087436   1558.0
           11264.0  b'RETL'  38.040001   52975.0  0.055494    950.0
           11363.0  b'SCIN'  23.760000   29995.0  0.014085   1500.0
           11407.0   b'LIT'  39.669998  510632.0  0.023478  27675.0
...                     ...        ...       ...       ...      ...
2018-06-29 93377.0  b'EIDO'  22.969999  901204.0  0.029122  14750.0
           93378.0  b'EIRL'  47.330002   49615.0  0.010893   1400.0
           93379.0  b'EPOL'  21.780001  436991.0  0.017282  11550.0
           93385.0  b'IBND'  33.820000   42089.0  0.011969   6800.0
           93421.0  b'PICB'  26.139999   26499.0  0.007322   5100.0

[240169 rows x 5 columns]

#5.  Create a column for the lagged return (i.e. one day return) for every ETF, and drop any null values.

In [0]:
# Multi-index where the first index is the ETF PERMNO since we want the lagged returns 
# for every ETF
etf2 = etf.set_index(['PERMNO','DATE'])
etf2.sort_index(inplace=True) 
# Create the lagged returns by grouping by ETF PERMNO and using the shift() method
etf2['LAGRET'] = etf2['RET'].groupby('PERMNO').shift()
# Switch order of the index given we now have the lagged returns for every ETF
etf3 = etf2.reorder_levels(['DATE','PERMNO']) 
etf3.sort_index(inplace=True) # We want time ordeed chronologically

etf3.dropna(inplace=True) # Drop every "NA"
etf3.head()

TICKER        PRC       VOL       RET   SHROUT    LAGRET
DATE       PERMNO                                                            
2018-01-03 10113.0  b'AADR'  60.930000   44385.0  0.010448   2850.0  0.024639
           11182.0  b'GASL'  28.400000  147624.0  0.028613   1558.0  0.087436
           11264.0  b'RETL'  38.020000   53995.0 -0.000526    950.0  0.055494
           11363.0  b'SCIN'  23.820000   16927.0  0.002525   1500.0  0.014085
           11407.0   b'LIT'  39.880001  836278.0  0.005294  27675.0  0.023478

#6. For every date, use pandas cut or qcut function to bin the lagged returns into quintiles (5 categories, hint: going to need a groupby and an apply method). That is, we’re breaking up stocks every day into quintiles.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html#pandas.qcut

In [0]:
# Let's create a qcut function that can split returns into quintiles
def quintiles(x):
    y = pd.qcut(x, q=5, labels=range(1,6))
    return y
# Now, we just group our returns by DATE and then apply our "quintiles" function
# to every return (assigning a quintile for every ETF)
etf3['QUINTILE'] = etf3['LAGRET'].groupby('DATE').apply(quintiles)

etf3

TICKER        PRC       VOL  ...   SHROUT    LAGRET  QUINTILE
DATE       PERMNO                                 ...                             
2018-01-03 10113.0  b'AADR'  60.930000   44385.0  ...   2850.0  0.024639         5
           11182.0  b'GASL'  28.400000  147624.0  ...   1558.0  0.087436         5
           11264.0  b'RETL'  38.020000   53995.0  ...    950.0  0.055494         5
           11363.0  b'SCIN'  23.820000   16927.0  ...   1500.0  0.014085         5
           11407.0   b'LIT'  39.880001  836278.0  ...  27675.0  0.023478         5
...                     ...        ...       ...  ...      ...       ...       ...
2018-06-29 93377.0  b'EIDO'  22.969999  901204.0  ...  14750.0 -0.013699         1
           93378.0  b'EIRL'  47.330002   49615.0  ...   1400.0 -0.005311         1
           93379.0  b'EPOL'  21.780001  436991.0  ...  11550.0 -0.008337         1
           93385.0  b'IBND'  33.820000   42089.0  ...   6800.0 -0.004468         1
           93421.0  b'PICB'  26.139999   26499.0  ...   5100.0 -0.002307         2

[237833 rows x 7 columns]

#7. Now, for every quintile, and on every date, compute the mean returns of every stock (grouped by quintile and date), and drop any null values.

In [0]:
# Computes portfolio returns
etfport_mean_returns = etf3.groupby(['QUINTILE','DATE'])['RET'].mean()
# Drop any null values
etfport_mean_returns.dropna(inplace=True)
# Portfolio Returns for every quintile, on every date. 
etfport_mean_returns

QUINTILE  DATE      
1         2018-01-03   -0.000875
          2018-01-04    0.000405
          2018-01-05    0.000827
          2018-01-08    0.000448
          2018-01-09    0.000868
                          ...   
5         2018-06-25   -0.014750
          2018-06-26   -0.003191
          2018-06-27   -0.007442
          2018-06-28   -0.003236
          2018-06-29    0.004787
Name: RET, Length: 620, dtype: float64

#8.  Note we can think of what we’re doing as creating a portfolio based on quintiles and adjusting every day. Compute some summary statistics for these portfolios (i.e. for the ETFs selected by quintile) using the .describe() method. More specifically, for every quintile, compute the summary statistics of the portfolios using .describe().

In [0]:
# Summary statistics of the mean return of our portfolios for every quintile 
etfport_mean_returns.groupby(['QUINTILE']).describe()

,count,mean,std,min,25%,50%,75%,max
QUINTILE,,,,,,,,
1,124.0,0.000812,0.008410,-0.044005,-0.001926,0.000872,0.005058,0.028111
2,124.0,0.000288,0.006658,-0.035076,-0.002115,0.000808,0.003674,0.020342
3,124.0,0.000032,0.006598,-0.031279,-0.002289,0.000698,0.004696,0.014254
4,124.0,-0.000406,0.006468,-0.021305,-0.002989,0.000197,0.003424,0.013789
5,124.0,-0.000926,0.007525,-0.028807,-0.003669,-0.000496,0.003104,0.015449


#9.  Compute the portfolio that every day buys some weight (you choose) of the highest quintile, and sells (shorts) some weight of the lowest quintile. When you do this, you should have a column or series of daily returns this “momentum” style portfolio. Compute the mean and standard deviation of this portfolio.

##First let's look at the ETFs in these quintiles to see what strategy we decide to employ.

In [0]:
etf3[etf3['QUINTILE'] == 1] 

TICKER         PRC        VOL  ...   SHROUT    LAGRET  QUINTILE
DATE       PERMNO                                   ...                             
2018-01-03 12065.0  b'SCHP'   55.340000   292223.0  ...  51750.0 -0.002526         1
           12289.0  b'CORP'  105.290497    30915.0  ...   8080.0 -0.003217         1
           12438.0  b'HYLD'   36.277199    48417.0  ...   4200.0 -0.001992         1
           12442.0  b'DUST'   22.639999  5488756.0  ...  11769.0 -0.072574         1
           12457.0  b'KBWP'   58.139999    25930.0  ...   1650.0 -0.021212         1
...                     ...         ...        ...  ...      ...       ...       ...
2018-06-29 93362.0  b'PSCI'   67.704597    11260.0  ...   1750.0 -0.005187         1
           93377.0  b'EIDO'   22.969999   901204.0  ...  14750.0 -0.013699         1
           93378.0  b'EIRL'   47.330002    49615.0  ...   1400.0 -0.005311         1
           93379.0  b'EPOL'   21.780001   436991.0  ...  11550.0 -0.008337         1
           93385.0  b'IBND'   33.820000    42089.0  ...   6800.0 -0.004468         1

[47782 rows x 7 columns]

In [0]:
etf3[etf3['QUINTILE'] == 5]

TICKER         PRC       VOL  ...   SHROUT    LAGRET  QUINTILE
DATE       PERMNO                                  ...                             
2018-01-03 10113.0  b'AADR'   60.930000   44385.0  ...   2850.0  0.024639         5
           11182.0  b'GASL'   28.400000  147624.0  ...   1558.0  0.087436         5
           11264.0  b'RETL'   38.020000   53995.0  ...    950.0  0.055494         5
           11363.0  b'SCIN'   23.820000   16927.0  ...   1500.0  0.014085         5
           11407.0   b'LIT'   39.880001  836278.0  ...  27675.0  0.023478         5
...                     ...         ...       ...  ...      ...       ...       ...
2018-06-29 93284.0  b'SOXL'  141.500000  586633.0  ...   4750.0  0.025253         5
           93336.0   b'UBR'   47.369999    1675.0  ...    217.0  0.063428         5
           93341.0   b'BIB'   57.200001   83099.0  ...   5950.0  0.012445         5
           93342.0  b'BICK'   27.455500    5526.0  ...   8150.0  0.010196         5
           93366.0  b'PSCH'  128.490005   77087.0  ...   5900.0  0.007437         5

[47520 rows x 7 columns]

## **DECISION**: Every day let's buy 100% of stock in the first (best) quintile and sell 100% of the stocks in the last (worst) quintile

In [0]:
# Essentially, every day, we just need to drop the ETFs in quintile 5
etf3.drop(etf3[etf3['QUINTILE'] == 5].index, inplace = True)
etf3

TICKER        PRC        VOL  ...    SHROUT    LAGRET  QUINTILE
DATE       PERMNO                                  ...                              
2018-01-03 11996.0  b'EMLC'  19.250000  5155454.0  ...  247000.0  0.010537         4
           12047.0  b'INXX'  16.160000    50212.0  ...    3700.0  0.005988         3
           12059.0  b'SCHR'  53.320000   102081.0  ...   21800.0  0.000000         2
           12064.0  b'SCHO'  49.990002   449647.0  ...   43450.0  0.000200         2
           12065.0  b'SCHP'  55.340000   292223.0  ...   51750.0 -0.002526         1
...                     ...        ...        ...  ...       ...       ...       ...
2018-06-29 93377.0  b'EIDO'  22.969999   901204.0  ...   14750.0 -0.013699         1
           93378.0  b'EIRL'  47.330002    49615.0  ...    1400.0 -0.005311         1
           93379.0  b'EPOL'  21.780001   436991.0  ...   11550.0 -0.008337         1
           93385.0  b'IBND'  33.820000    42089.0  ...    6800.0 -0.004468         1
           93421.0  b'PICB'  26.139999    26499.0  ...    5100.0 -0.002307         2

[190313 rows x 7 columns]

In [0]:
# Computes portfolio returns
new_etfport_mean_returns = etf3.groupby(['QUINTILE','DATE'])['RET'].mean()
# Drop any null values
new_etfport_mean_returns.dropna(inplace=True)
# Portfolio Returns for every quintile, on every date. 
new_etfport_mean_returns

QUINTILE  DATE      
1         2018-01-03   -0.000875
          2018-01-04    0.000405
          2018-01-05    0.000827
          2018-01-08    0.000448
          2018-01-09    0.000868
                          ...   
4         2018-06-25   -0.010171
          2018-06-26    0.000047
          2018-06-27   -0.008502
          2018-06-28    0.000144
          2018-06-29    0.004480
Name: RET, Length: 496, dtype: float64

##Mean Return of this Portfolio

In [0]:
np.mean(etf3['RET'])

0.000176177077950922

#Standard Deviation of this Portfolio's Returns

In [0]:
np.std(etf3['RET'])

0.012884273231471264

##Summary Statistics of Returns for this Portfolio by Quintile.

In [0]:
new_etfport_mean_returns.groupby(['QUINTILE']).describe()

,count,mean,std,min,25%,50%,75%,max
QUINTILE,,,,,,,,
1,124.0,0.000812,0.008410,-0.044005,-0.001926,0.000872,0.005058,0.028111
2,124.0,0.000288,0.006658,-0.035076,-0.002115,0.000808,0.003674,0.020342
3,124.0,0.000032,0.006598,-0.031279,-0.002289,0.000698,0.004696,0.014254
4,124.0,-0.000406,0.006468,-0.021305,-0.002989,0.000197,0.003424,0.013789
